In [140]:
import requests
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from shapely import wkt
from shapely.geometry import Polygon

In [141]:
# Function to extract geometry information from json and save it in Pandas DataFrame
def json_to_dataframe(json_data):
    # Extract the features from the JSON data
    features = json_data['features']
    
    # Extract the geometry information
    data = [feature['geometry'] for feature in features]
    
    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    
    # Extract the polygon from the data
    df['geometry'] = df['rings'].apply(lambda x: x[0])
    
    return df

In [142]:
# Function to calculate the centroid of polygons
def get_centroid(coords):
    polygon = Polygon(coords)
    return polygon.centroid

In [143]:
# Function to get the geometry information of the parcel data from ArcGIS service
def get_parcel_data(state, bpoly):
    # Get the URL of the state parcel data
    df = pd.read_csv('us_states_parcel_data.csv') # We might want to change the directory to the spreadsheet
    url = df.loc[df['State'] == state, 'URL'].values[0]
    
    # Get the coordinates of the bounding box for input polygon bpoly
    bbox = bpoly.bounds
    
    # Construct the query URL to get the parcel data within the bounding box
    query = (url + '/query?' +
             f'geometry={bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}' +
             '&geometryType=esriGeometryEnvelope&inSR=4326' + 
             '&spatialRel=esriSpatialRelIntersects' +
             '&outSR=4326' +
             '&returnGeometry=true&f=json')
    
    # Set up a session with retries
    s = requests.Session()
    retries = Retry(total=5, 
                    backoff_factor=0.1,
                    status_forcelist=[500, 502, 503, 504])
    s.mount('https://', HTTPAdapter(max_retries=retries))

    # Send the request and get the response
    r = s.get(query)
    
    # Get the parcel data in JSON format
    parcel_data = r.json()
    
    # Extract the geometry information from the JSON file and store in the csv format
    df = json_to_dataframe(parcel_data)

    # Calculate the centroid for each parcel
    df['centroid'] = df['geometry'].apply(lambda x: get_centroid(x))

    # Extract x (longitude) and y (latitude) coordinates
    df['xcoord'] = df['centroid'].apply(lambda point: point.x)
    df['ycoord'] = df['centroid'].apply(lambda point: point.y)
    
    return df

In [144]:
### CHANGE HERE (Alameda Island Example)
from shapely.geometry import box, shape
# Define the bounding box for Alameda Island
bbox = box(-89.5134, 42.9986, -89.2751, 43.1337)

# Get the parcel data for Alameda Island in JSON format
parcel_data = get_parcel_data('Wisconsin', bbox)

parcel_data.head()

,rings,geometry,centroid,xcoord,ycoord
0,"[[[-89.3100241841761, 43.1306224991847], [-89....","[[-89.3100241841761, 43.1306224991847], [-89.3...",POINT (-89.30959805303594 43.1322607232037),-89.309598,43.132261
1,"[[[-89.2905107695329, 43.1185509113959], [-89....","[[-89.2905107695329, 43.1185509113959], [-89.2...",POINT (-89.29051503471022 43.118735320929126),-89.290515,43.118735
2,"[[[-89.4861498230017, 43.0190649186565], [-89....","[[-89.4861498230017, 43.0190649186565], [-89.4...",POINT (-89.48614136612021 43.019448467045386),-89.486141,43.019448
3,"[[[-89.3604115756996, 43.1124261871869], [-89....","[[-89.3604115756996, 43.1124261871869], [-89.3...",POINT (-89.36063380914224 43.112496271658095),-89.360634,43.112496
4,"[[[-89.3481879421997, 43.1068914568851], [-89....","[[-89.3481879421997, 43.1068914568851], [-89.3...",POINT (-89.34826460165316 43.10705659096423),-89.348265,43.107057


In [11]:
parcel_data.to_csv('parcel.csv')